In [1]:
import gym
import numpy as np
import time

In [2]:
def discretize(min, max, num):
    return np.linspace(min, max, num + 1)[1:-1]

In [3]:
def digitize_state(observation, fineness):
    cart_x, cart_v, pole_th, pole_v = observation
    digitized_state = [
        np.digitize(cart_x,  bins=discretize(-2.4, 2.4, fineness)),
        np.digitize(cart_v,  bins=discretize(-3.0, 3.0, fineness)),
        np.digitize(pole_th, bins=discretize(-0.5, 0.5, fineness)),
        np.digitize(pole_v,  bins=discretize(-2.0, 2.0, fineness))
    ]
    return sum([x * (fineness**i) for i, x in enumerate(digitized_state)])

In [4]:
def update_qtable(q_table, current_status, action, next_status, current_reward):
    epsilon = 0.5
    next_reward = np.max(q_table[next_status])
    q_table[current_status, action] = (1 - epsilon) * q_table[current_status, action] + \
                                        epsilon * (current_reward + next_reward)
    return q_table

In [5]:
def calc_action(status, q_table):
    return 0 if q_table[status][0] > q_table[status][1] else 1

In [6]:
def calc_reward(observation, done, current_step, max_steps):
    if done:
        return 200 if current_step + 1 == max_steps else -200
    else:
        return 10 - sum([abs(observation[0]), abs(observation[1]), abs(observation[2]), abs(observation[3])])

In [7]:
episode_rewards = []

In [8]:
def main():
    FINENESS = 8
    EPISODES  = 1000
    MAX_STEPS = 200
      
    env = gym.make('CartPole-v1')
    q_table = np.random.rand(FINENESS**env.observation_space.shape[0], env.action_space.n)
    
    for episode in range(EPISODES):
        observation = env.reset()
        episode_reward = 0
        
        for step in range(MAX_STEPS):
            # do action
            current_status = digitize_state(observation, FINENESS)
            action = calc_action(current_status, q_table)
            observation, reward, done, info = env.step(action)
            episode_reward += reward
            
            # calc reward
            reward = calc_reward(observation, done, step, MAX_STEPS)
            
            # update q_table
            next_status = digitize_state(observation, FINENESS)
            q_table = update_qtable(q_table, current_status, action, next_status, reward)
            
            # render
            if (episode + 1) % 100 == 0:
                env.render()
                time.sleep(0.01)
            
            if done:
                break
#         print("episode_reward: {}".format(episode_reward))
        episode_rewards.append(episode_reward)
        
    env.close()

main()

In [9]:
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
x = np.arange(len(episode_rewards))
y = np.array(episode_rewards)

plt.plot(x, y)
plt.show()